![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.svg)
# Pipeline Notebook - AutoAI Notebook v1.22.4

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimal results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.10 and scikit-learn 1.1.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data with train and test data split](#read)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
&nbsp;&nbsp;[Predict](#predict)<br>
**[Store the model](#saving)**<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watsonx-ai,
 - autoai-ts-libs,
 - scikit-learn


In [ ]:
!pip install ibm-watsonx-ai | tail -n 1
!pip install -U 'autoai-ts-libs>=3.0.0,<4.0' | tail -n 1
!pip install -U scikit-learn==1.1.1 | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [ ]:
from ibm_watsonx_ai.helpers import DataConnection
from ibm_watsonx_ai.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='0561ee0c-dadf-4264-abb5-cceb4f0cdd04'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/1e044060-954b-4980-b790-2a64bb613f30/wml_data/c68e6702-b8cb-4065-b111-8c4ce4edfe27/data/autoai-tsad',
        model_location='auto_ml/1e044060-954b-4980-b790-2a64bb613f30/wml_data/c68e6702-b8cb-4065-b111-8c4ce4edfe27/data/autoai-tsad/model.zip',
        training_status='auto_ml/1e044060-954b-4980-b790-2a64bb613f30/wml_data/c68e6702-b8cb-4065-b111-8c4ce4edfe27/training-status.json'
    )
)

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
    prediction_type='timeseries_anomaly_prediction',
    feature_columns=['risk_score'],
    csv_separator=',',
    holdout_size=0.2,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    timestamp_column_name=-1,
    pipeline_types=['PointwiseBoundedHoltWintersAdditive', 'PointwiseBoundedBATS', 'PointwiseBoundedBATSForceUpdate', 'WindowNN', 'WindowLOF', 'WindowPCA'],
    max_number_of_estimators=6,
    scoring='average_precision',
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='d216515b-da89-4e73-bb5b-fe44ca0014b4'
)

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [ ]:
api_key = 'PUT_YOUR_APIKEY_HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [ ]:
from ibm_watsonx_ai import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(wml_client)



<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data with train and test data split

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`

In [ ]:
train_X, test_X, train_y, test_y = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=True)

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_ts_libs.anomaly_detection.estimators.api.base import TSADPipeline
from autoai_ts_libs.anomaly_detection.estimators.watfore.pointwise_bounded_ad import (
    PointwiseBoundedAnomalyDetector,
)

#### Pipeline definition.

In [ ]:
pointwise_bounded_anomaly_detector = PointwiseBoundedAnomalyDetector(
    algorithm="bats",
    run_mode="integration",
    target_column_indices=[0],
    ts_icol_loc=[],
    update_with_anomaly=False,
)
pipeline = TSADPipeline(
    steps=[("PointwiseBoundedBATS", pointwise_bounded_anomaly_detector)],
    memory=None,
    verbose=False,
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from autoai_ts_libs.utils.metrics import get_scorer

scorer = get_scorer(experiment_metadata["scoring"])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(train_X.values)

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

<a id="predict"></a>
## Predict

Predict anomalies with the new observations. The predicted value for a record will take values -1 or 1, where -1 indicates the record is an anomaly, while 1 indicates the record is normal data.

In [ ]:
import numpy

simulated_records = 5
new_obs = numpy.random.randn(simulated_records,test_X.values.shape[1])
pipeline.predict(new_obs)

<a id="saving"></a>
## Store the model

In this section you will learn how to store the retrained model.

In [ ]:
model_metadata = {
    wml_client.repository.ModelMetaNames.NAME: 'P1 - Pretrained AutoAI pipeline'
}

stored_model_details = wml_client.repository.store_model(model=pipeline, meta_props=model_metadata, experiment_metadata=experiment_metadata)

Inspect the stored model details.

In [ ]:
stored_model_details

<a id="deployment"></a>
## Create online deployment

You can use commands bellow to promote the model to space and create online deployment (web service).

<a id="working_spaces"></a>
### Working with spaces

In this section you will specify a deployment space for organizing the assets for deploying and scoring the model. If you do not have an existing space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create a new space, following these steps:

- Click **New Deployment Space**.
- Create an empty space.
- Select Cloud Object Storage.
- Select Watson Machine Learning instance and press **Create**.
- Copy `space_id` and paste it below.

**Tip**: You can also use the API to prepare the space for your work. Learn more [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign or update space ID below.

#### Prepare online deployment

#### Test online deployment

<a id="cleanup"></a>
### Deleting deployment
You can delete the existing deployment by calling the `wml_client.deployments.delete(deployment_id)` command.
To list the existing web services, use `wml_client.deployments.list()`.

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/cpd/overview/relnotes-4.0.html) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___